# Crash Dummies: Identifying High Risk Areas from NYC Traffic Conditions
## *Data Cleaning*

### Table of Contents

* [Topic 1](#topic-1)
* [Topic 2](#topic-2)

#### Import Libraries & Read in Data

In [25]:
## STANDARD IMPORTS
import pandas as pd 
import numpy as np

# MAPS & GEO LOCATION
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
from shapely.geometry import Point, Polygon

# ## visualizations
# import matplotlib.pyplot as plt
# import seaborn as sns
# ## preprocessing
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler, PolynomialFeatures
# from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.dummy import DummyClassifier
# ## modeling
# from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge, ElasticNet
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
# from sklearn.svm import SVC, SVR
# from sklearn.naive_bayes import MultinomialNB
# ## trees
# from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
# from sklearn.ensemble import BaggingClassifier, BaggingRegressor, RandomForestClassifier, ExtraTreesClassifier, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, AdaBoostClassifier, GradientBoostingRegressor
# ## NLP
# from sklearn.feature_extraction.text import CountVectorizer
# ## analysis
# from sklearn.metrics import confusion_matrix, plot_confusion_matrix, accuracy_score, make_scorer, f1_score, mean_squared_error

# ## options
# import sklearn

pd.options.display.max_rows = 4000
pd.options.display.max_columns = 100
pd.set_option('max_colwidth', 100)

In [2]:
### read in data
df = pd.read_csv('~/git_repos/ga_repos/05_project/data_large/collisions.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head(2)

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,10/09/2020,10:12,NaN,NaN,40.683933,-73.87119,"(40.683933, -73.87119)",HEMLOCK STREET,NaN,NaN,1.0,0.0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,4356254,NaN,NaN,NaN,NaN,NaN
1,10/09/2020,15:40,NaN,NaN,NaN,NaN,NaN,HUTCHINSON RIVER PARKWAY,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4356784,Sedan,NaN,NaN,NaN,NaN


In [4]:
df.shape

(1720022, 29)

## Check for Missing Data <a class="anchor" id="topic-1"></a>
<hr/>

In [5]:
df.isna().sum()

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                           526355
ZIP CODE                          526564
LATITUDE                          205510
LONGITUDE                         205510
LOCATION                          205510
ON STREET NAME                    343816
CROSS STREET NAME                 596739
OFF STREET NAME                  1468574
NUMBER OF PERSONS INJURED             17
NUMBER OF PERSONS KILLED              31
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       4733
CONTRIBUTING FACTOR VEHICLE 2     239060
CONTRIBUTING FACTOR VEHICLE 3    1606331
CONTRIBUTING FACTOR VEHICLE 4    1695836
CONTRIBUTING FACTOR VEHICLE 5    1713759
COLLISION_ID                           0
VEHICLE TYPE COD

In [6]:
# Don't have a location, but DOES have a street name & DOES have a cross street 
((df['ON STREET NAME'].isna() == False) & (df['LOCATION'].isna() == True) & (df['CROSS STREET NAME'].isna() == False)).sum()

128262

In [7]:
# don't have a street name or location GPS
((df['ON STREET NAME'].isna() == True) & (df['LOCATION'].isna() == True)).sum()

52949

We need really only need GPS location for every crash so we'll just drop whatever missing values we don't need.

In [8]:
# Removing NaNs from  ['LATITUDE','LONGITUDE']  row 
df_clean = df.dropna(subset=['LATITUDE','LONGITUDE']).copy()

## Removing Outliers <a class="anchor" id="topic-2"></a>
<hr/>

There are a bunch of GPS coordinates that are out of the range of NYC. These have been removed from the dataset.

In [9]:
df_clean = df_clean[(df_clean['LONGITUDE'] != 0)]
df_clean = df_clean[(df_clean['LATITUDE'] > 38) & (df_clean['LATITUDE'] < 41)] #.sum() # > 41
df_clean = df_clean[(df_clean['LONGITUDE'] < -50) & (df_clean['LONGITUDE'] > -74.6)]

In [10]:
df_clean.shape

(1513092, 29)

In [12]:
df_clean.head(3)

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,10/09/2020,10:12,NaN,NaN,40.683933,-73.871190,"(40.683933, -73.87119)",HEMLOCK STREET,NaN,NaN,1.0,0.0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,4356254,NaN,NaN,NaN,NaN,NaN
3,10/08/2020,10:46,NaN,NaN,40.827210,-73.920700,"(40.82721, -73.9207)",EAST 162 STREET,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,NaN,4355967,Sedan,AMBU,NaN,NaN,NaN
4,10/08/2020,19:24,NaN,NaN,40.799310,-73.943245,"(40.79931, -73.943245)",PARK AVENUE,NaN,NaN,1.0,0.0,1,0,0,0,0,0,Driver Inexperience,NaN,NaN,NaN,NaN,4356130,Sedan,NaN,NaN,NaN,NaN


## Formatting Datetime <a class="anchor" id="topic-2"></a>
<hr/>

In [13]:
## create unified date time column
df_clean['DATE'] = df_clean['CRASH DATE'] + ' ' + df_clean['CRASH TIME']

In [14]:
df_clean['DATE'] = pd.to_datetime(df_clean['DATE'])

In [15]:
df_clean.head(2)

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,DATE
0,10/09/2020,10:12,NaN,NaN,40.683933,-73.87119,"(40.683933, -73.87119)",HEMLOCK STREET,NaN,NaN,1.0,0.0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,4356254,NaN,NaN,NaN,NaN,NaN,2020-10-09 10:12:00
3,10/08/2020,10:46,NaN,NaN,40.827210,-73.92070,"(40.82721, -73.9207)",EAST 162 STREET,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,NaN,4355967,Sedan,AMBU,NaN,NaN,NaN,2020-10-08 10:46:00


In [17]:
### set as index
df_clean.set_index('DATE', inplace=True)
### sort index
df_clean.sort_index(inplace=True)

In [18]:
### export copy of whole data set in case we need it in future
# df_clean.to_csv('../data/crash_features_clean.csv', index = False)

## Reducing Data Set
<hr/>

Because the data set is still too large for GitHub, we'll separate out the data by year.

In [48]:
### creates CSV file for dataframe and list of years.
def crash_year(df, list_years):
    for year in list_years:
        data = df.loc[year] 
        data.to_csv(f'../data/crash_features_{year}.csv')
    
    pass  

In [49]:
### create needed years
years = ['2018', '2019', '2020']
crash_year(df_clean, years)

## Reduce to Lincoln Tunnel Area <a class="anchor" id="topic-2"></a>
<hr/>

We chose to reduce our data to the lincoln tunnel area since we can also obtain complimentary speed data for this area. We'll reduce our crash data to this area for crashes in 2019.

In [80]:
### read in 2019 data
df = pd.read_csv('../data/crash_features_2019.csv')

In [71]:
df.head(2)

,DATE,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-01-01 00:00:00,01/01/2019,0:00,BROOKLYN,11234.0,40.621563,-73.923996,"(40.621563, -73.923996)",NaN,NaN,5315 AVENUE M,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4060629,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
1,2019-01-01 00:00:00,01/01/2019,0:00,NaN,NaN,40.639210,-74.171260,"(40.63921, -74.17126)",NaN,NaN,55 HOLLAND AVENUE,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Backing Unsafely,NaN,NaN,NaN,4060523,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN


In [81]:
### creates dataframe with geometry data for maps
df = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.LONGITUDE, df.LATITUDE))

In [82]:
### reduce columns - only need 
red_cols = [
    'DATE', 
    'CRASH DATE', 
    'CRASH TIME',
    'LATITUDE', 
    'LONGITUDE', 
    'LOCATION', 
    'BOROUGH', 
    'ZIP CODE',
    'CONTRIBUTING FACTOR VEHICLE 1',
    'geometry']

df = df[red_cols]

### Filter data

In [83]:
### Coordinates for lincoln tunnel area of interest
lincoln_tunnel_coords = [(-74.000009, 40.763306),( -73.985957,40.757391),( -73.992627, 40.747933), (-74.006919,40.754636)]

In [84]:
# apply the contains method to to polygon and points 
row_indices=[]
### create polygon shape for filtering data
poly = Polygon(lincoln_tunnel_coords)
for i, j in df.iterrows():
    if (poly.contains(j['geometry'])):
        row_indices.append(i)
    
print('Total Number of Crashes: ', len(row_indices))

Total Number of Crashes:  2560


In [85]:
tunnel_crashes = df[df.index.isin(row_indices)]

In [86]:
tunnel_crashes.head()

,DATE,CRASH DATE,CRASH TIME,LATITUDE,LONGITUDE,LOCATION,BOROUGH,ZIP CODE,CONTRIBUTING FACTOR VEHICLE 1,geometry
73,2019-01-01 02:30:00,01/01/2019,2:30,40.749706,-73.991570,"(40.749706, -73.99157)",MANHATTAN,10001.0,Passing or Lane Usage Improper,POINT (-73.99157 40.74971)
109,2019-01-01 04:23:00,01/01/2019,4:23,40.761500,-73.997826,"(40.7615, -73.997826)",NaN,NaN,Driver Inattention/Distraction,POINT (-73.99783 40.76150)
122,2019-01-01 04:50:00,01/01/2019,4:50,40.751026,-73.996620,"(40.751026, -73.99662)",MANHATTAN,10001.0,Unspecified,POINT (-73.99662 40.75103)
337,2019-01-01 19:03:00,01/01/2019,19:03,40.754055,-73.995830,"(40.754055, -73.99583)",MANHATTAN,10018.0,Other Vehicular,POINT (-73.99583 40.75406)
347,2019-01-01 19:45:00,01/01/2019,19:45,40.758533,-73.988850,"(40.758533, -73.98885)",NaN,NaN,Turning Improperly,POINT (-73.98885 40.75853)


In [87]:
### Export to csv
tunnel_crashes.to_csv('../data/crashes_lincoln_tunnel.csv', index=False)

## Crash Numbers by Day <a class="anchor" id="topic-2"></a>
<hr/>

We also need a data set that just takes the total number of crashes per day in the Lincoln Tunnel area.

In [112]:
crashes_daily = tunnel_crashes[['CRASH DATE', 'LOCATION']].copy()
### convert crash date to datetime object
crashes_daily['CRASH DATE'] = pd.to_datetime(crashes_daily['CRASH DATE'])
### set as index
crashes_daily.set_index('CRASH DATE', inplace=True)

crashes_daily.rename(columns={'LOCATION': 'CRASHES'}, inplace=True)

### get counts for each day
crashes_daily_counts = crashes_daily.groupby(crashes_daily.index.date).count()

In [113]:
crashes_daily_counts.head()

,CRASHES
2019-01-01,5
2019-01-02,5
2019-01-03,4
2019-01-04,8
2019-01-05,1


In [114]:
### export to CSV
crashes_daily_counts.to_csv('../data/daily_crash_counts_lincoln_tunnel.csv')